In [8]:
from graphviz import Digraph

def create_model_diagram():
    dot = Digraph(comment='SelfGuidedTSDiff Architecture')

    # General styling
    dot.attr(rankdir='TB', splines='spline', bgcolor='white')
    
    # Nodes for SelfGuidedTSDiff and TSDiff
    dot.node('SGTSDiff', 'SelfGuidedTSDiff', shape='box', style='rounded, filled', fillcolor='lightgrey')
    dot.node('TSDiff', 'TSDiff', shape='box', style='rounded, filled', fillcolor='lightgrey')

    # TSDiff Architecture components split into layers
    dot.node('InitConv', 'Initial Conv1D Layer', shape='box', style='filled', fillcolor='lightblue', rank='same')
    dot.node('ResBlocks', 'N Residual Blocks', shape='box', style='filled', fillcolor='lightcoral', rank='same')
    dot.node('TransLayer', 'Transformer Layer', shape='box', style='filled', fillcolor='lightgreen', rank='same')
    dot.node('OutConv', 'Output Conv1D Layer', shape='box', style='filled', fillcolor='lightblue', rank='same')

    # Observation Self-Guidance components
    dot.node('Xt', 'x^t', shape='ellipse', style='filled', fillcolor='lightyellow')
    dot.node('Xt1', 'x^{t-1}', shape='ellipse', style='filled', fillcolor='lightyellow')
    dot.node('Eps', 'e(x^t, t)', shape='ellipse', style='filled', fillcolor='lightyellow')
    dot.node('Yobs', 'y_{obs}', shape='ellipse', style='filled', fillcolor='lightyellow')

    # Grouping TSDiff layers
    with dot.subgraph(name='cluster_TSDiff') as c:
        c.attr(style='dashed', label='TSDiff Architecture', color='black')
        c.node('InitConv')
        c.node('ResBlocks')
        c.node('TransLayer')
        c.node('OutConv')
        c.edge('InitConv', 'ResBlocks', label='', color='black')
        c.edge('ResBlocks', 'TransLayer', label='', color='black')
        c.edge('TransLayer', 'OutConv', label='', color='black')

    # External connections showing integration with Observation Self-Guidance
    dot.edge('Xt', 'InitConv', label='Input x^t', color='black')
    dot.edge('OutConv', 'Eps', label='Output e(x^t, t)', color='black')
    dot.edge('Eps', 'Xt1', label='', color='black')
    dot.edge('Xt1', 'Yobs', label='', color='black')
    dot.edge('Yobs', 'Eps', label='Feedback', color='black', style='dashed')

    # Connect TSDiff to Self-GuidedTSDiff
    dot.edge('SGTSDiff', 'TSDiff', label='Calls TSDiff', color='black')
    dot.edge('TSDiff', 'OutConv', label='', color='black')
    
    # Self-guidance gradient flow
    dot.node('GradCalc', 'Gradient Calculation', shape='ellipse', style='filled', fillcolor='lightyellow')
    dot.edge('Yobs', 'GradCalc', label='Gradient Input', color='black', style='dashed')
    dot.edge('GradCalc', 'InitConv', label='Gradient Adjustment', color='black', style='dashed')

    return dot

# Render the diagram
diagram = create_model_diagram()
diagram.render('output/self_guided_tsdiff_diagram', format='png', cleanup=True)
diagram.view()


'output\\self_guided_tsdiff_diagram.pdf'

In [32]:
from graphviz import Digraph

# Create the Block Diagram graph
block_diagram = Digraph(comment='TimeGAN Block Diagram', format='png')
block_diagram.attr(rankdir='LR', splines='polyline', concentrate='true')
block_diagram.attr(label='Block Diagram', style='dashed', labelloc='t', center='true')
block_diagram.node('Real Sequences', 'Real Sequences\nε S × T X', shape='ellipse', style='filled', color='gray')
block_diagram.node('Random Vectors', 'Random Vectors\nε ZS × T Z', shape='ellipse', style='filled', color='gray')
block_diagram.node('Latent Codes', 'Latent Codes\nε HS × T H', shape='ellipse', style='filled', color='gray')
block_diagram.node('Reconstruction', 'Reconstruction\nε S × T X', shape='ellipse', style='filled', color='gray')
block_diagram.node('Classifications', 'Classifications\nε [0, 1]', shape='ellipse', style='filled', color='gray')

# Add edges for the block diagram
block_diagram.edge('Real Sequences', 'Latent Codes', label='Embeder')
block_diagram.edge('Random Vectors', 'Latent Codes', label='Generate')
block_diagram.edge('Latent Codes', 'Reconstruction', label='Recover')
block_diagram.edge('Latent Codes', 'Classifications', label='Discriminate')

# Add loss labels
block_diagram.node('Unsupervised Loss', 'Unsupervised Loss\nLearn distribution\np(S, X) directly', shape='note', color='orange')
block_diagram.node('Supervised Loss', 'Supervised Loss\nLearn conditionals\np(X|S)', shape='note', color='orange')
block_diagram.node('Reconstruction Loss', 'Reconstruction Loss\nProvide Latent\nEmbedding Space', shape='note', color='orange')

# Connect losses
block_diagram.edge('Classifications', 'Unsupervised Loss')
block_diagram.edge('Latent Codes', 'Supervised Loss')
block_diagram.edge('Reconstruction', 'Reconstruction Loss')

# Render the Block Diagram
block_diagram_output_path = './timegan_block_diagram'
block_diagram.render(block_diagram_output_path, cleanup=True)

# Create the Training Scheme graph
training_scheme = Digraph(comment='TimeGAN Training Scheme', format='png')
training_scheme.attr(rankdir='LR', splines='polyline', concentrate='true')
training_scheme.attr(label='Training Scheme', style='dashed', labelloc='t', center='true')
training_scheme.node('Input', 'Input\ns, x', shape='ellipse')
training_scheme.node('Latent', 'Latent\nhS, h', shape='ellipse')
training_scheme.node('Noise', 'Noise\nzS, z', shape='ellipse')
training_scheme.node('Reconstructed', 'Reconstructed\ns, x', shape='ellipse')
training_scheme.node('Output', 'Output\nyS, y', shape='ellipse')

# Training edges
training_scheme.node('Embedder', 'Embedder\ne', shape='box', style='filled', color='pink')
training_scheme.node('Recoverer', 'Recoverer\nr', shape='box', style='filled', color='pink')
training_scheme.node('Generator', 'Generator\ng', shape='box', style='filled', color='lightblue')
training_scheme.node('Discriminator', 'Discriminator\nd', shape='box', style='filled', color='lightblue')

# Connect nodes for training scheme
training_scheme.edge('Input', 'Embedder', label='∂LR/∂θe')
training_scheme.edge('Embedder', 'Latent')
training_scheme.edge('Latent', 'Generator', label='∂LS/∂θg')
training_scheme.edge('Noise', 'Generator')
training_scheme.edge('Generator', 'Output')
training_scheme.edge('Output', 'Discriminator', label='∂LU/∂θd')
training_scheme.edge('Discriminator', 'Latent', label='∂LU/∂θg', dir='back')

training_scheme.edge('Latent', 'Recoverer', label='∂LR/∂θr')
training_scheme.edge('Recoverer', 'Reconstructed')

# Render the Training Scheme
training_scheme_output_path = './timegan_training_scheme'
training_scheme.render(training_scheme_output_path, cleanup=True)

print(f"Graph code generated. Rendered file paths: {block_diagram_output_path}.png, {training_scheme_output_path}.png")


Graph code generated. Rendered file paths: ./timegan_block_diagram.png, ./timegan_training_scheme.png


In [5]:
from graphviz import Digraph

# Refining the detailed architecture graph to match the exact style of the provided diffusion model diagram

# Initialize Digraph for GAN architecture
gan_arch_tsdiff_style = Digraph(name="GAN_Styled_Model_Architecture", format="png")
gan_arch_tsdiff_style.attr(rankdir="TB")

# Main nodes for GAN architecture
gan_arch_tsdiff_style.node("Photometry_Input", "Photometry Input", shape="ellipse", style="filled", fillcolor="lightyellow")
gan_arch_tsdiff_style.node("Embedding", "Embedding Network", shape="box", style="filled", fillcolor="lightblue")
gan_arch_tsdiff_style.node("Generator", "Generator", shape="box", style="filled", fillcolor="lightcoral")
gan_arch_tsdiff_style.node("Generated_Spectra", "Generated Spectra (Output)", shape="ellipse", style="filled", fillcolor="lightyellow")
gan_arch_tsdiff_style.node("Discriminator", "Discriminator", shape="box", style="filled", fillcolor="lightgrey")
gan_arch_tsdiff_style.node("Real_Spectra", "Real Spectra", shape="ellipse", style="filled", fillcolor="lightyellow")
gan_arch_tsdiff_style.node("Loss_Calculation", "Loss Calculation", shape="ellipse", style="filled", fillcolor="lightyellow")

# Additional nodes and connections to mimic the detailed style with feedback and loss calculation
gan_arch_tsdiff_style.node("Gradient_Input", "Gradient Input", shape="ellipse", style="filled", fillcolor="lightyellow")
gan_arch_tsdiff_style.node("Gradient_Adjust", "Gradient Adjustment", shape="ellipse", style="filled", fillcolor="lightyellow")

# Connections within GAN architecture
gan_arch_tsdiff_style.edge("Photometry_Input", "Embedding")
gan_arch_tsdiff_style.edge("Embedding", "Generator")
gan_arch_tsdiff_style.edge("Generator", "Generated_Spectra")
gan_arch_tsdiff_style.edge("Generated_Spectra", "Discriminator")
gan_arch_tsdiff_style.edge("Real_Spectra", "Discriminator", label="Real vs. Generated")
gan_arch_tsdiff_style.edge("Discriminator", "Loss_Calculation", label="Adversarial Loss")
gan_arch_tsdiff_style.edge("Loss_Calculation", "Gradient_Input", label="Gradient Input", style="dashed")
gan_arch_tsdiff_style.edge("Gradient_Input", "Gradient_Adjust", style="dashed")
gan_arch_tsdiff_style.edge("Gradient_Adjust", "Generator", label="Gradient Adjustment", style="dashed")

# Display the GAN styled architecture graph
gan_arch_tsdiff_style.view()

'GAN_Styled_Model_Architecture.gv.png'